# Download/Pre-process CFS forecast data
Lindsay Fitzpatrick
ljob@umich.edu
08/28/2024
Updated: 02/17/2024

This script reads downloads CFS forecast data from the AWS as grib2 files. It then opens the grib2 files, calculates total basin, lake, and land, precipitation, evaporation, and average 2m air temperature. These calculations are then added to the running CSV files. This script needs the following files:

- GL_mask.nc
- CFS_EVAP_forecasts_Avgs_MM.csv
- CFS_PCP_forecasts_Avgs_MM.csv
- CFS_TMP_forecasts_Avgs_K.csv

In [2]:
from datetime import datetime, timedelta
import os
import sys
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import cfgrib
import pandas as pd
import netCDF4 as nc
import numpy as np
import shutil
import urllib.request
from bs4 import BeautifulSoup
import requests
import calendar
import csv
import time

## User Inputs

In [17]:
# Path to download data to
download_dir = '/Users/fitzpatrick/Desktop/Data/CFS/'
#download_dir = '/Users/fitzpatrick/Desktop/cnbs-predictor-1/data/input/'

# Input directory
dir = '/Users/fitzpatrick/Desktop/Data/cnbs/'
#dir = '/Users/fitzpatrick/Desktop/cnbs-predictor-1/data/input/'

# Location of the mask file
mask_file = dir + 'mask_0.31.nc'

# Location of existing CSV files or path/name to new CSV files
tmp_file = dir + 'CFS_TMP_forecasts_Avgs_K_9999.csv'
evap_file = dir + 'CFS_EVAP_forecasts_Avgs_MM_9999.csv'
pcp_file = dir + 'CFS_PCP_forecasts_Avgs_MM_9999.csv'

# Where would you like to pull the data from?
source = 'aws' # 'aws' or 'ncei'

# Do you need to download CFS data?
download_cfs = 'no'

# Do you want to process the CFS data and 
process_cfs = 'yes'

# Delete grib files after processing and saving data?
delete_files = 'no'

# Auto 'yes' will open the existing files, pull the last date to determine the start date and yesterday becomes the end date in
# order to make the csvs up-to-date
auto = 'no'

start_date = '2024-08-13'
end_date = '2025-02-01'

Presets

These shouldn't change unless the location changes for CFS data or the user wants different files (products specifies the prefix of the files. Different files contain different variables) or a specific forecast (utc specifies the forecast time).

In [3]:
## Presets ##
products = ['pgb','flx']
utc = ['00','06','12','18']

# Define mask variables
mask_variables = ['eri_lake','eri_land',
                 'ont_lake','ont_land',
                 'mih_lake','mih_land',
                 'sup_lake','sup_land']

#AWS bucket name to locate the CFS forecast
bucket_name = 'noaa-cfs-pds'

## Define Functions

This function goes to the AWS site and downloads the needed CFS files for a given forecast day.

In [4]:
def download_grb2_aws(product, bucket_name, url_path, download_dir):
    """
    Download the CFS forecast from AWS

    Parameters:
    - product: 'flx' or 'pgb'
    - bucket_name: for CFS data it is 'noaa-cfs-pds'
    - url_path: the url path to data
    - download_dir: location to download data to
    """
    num_files_downloaded = 0

    # Create a boto3 client for S3
    s3_config = Config(signature_version=UNSIGNED)
    s3 = boto3.client('s3', config=s3_config)

    # List all objects in the specified folder path
    continuation_token = None
    objects = []

    # Use a loop to handle pagination
    while True:
        list_objects_args = {'Bucket': bucket_name, 'Prefix': url_path}
        if continuation_token:
            list_objects_args['ContinuationToken'] = continuation_token

        list_objects_response = s3.list_objects_v2(**list_objects_args)

        objects.extend(list_objects_response.get('Contents', []))

        if not list_objects_response.get('IsTruncated', False):
            break

        continuation_token = list_objects_response.get('NextContinuationToken')

    # Iterate over each object and download if it ends with '.grb2'
    for obj in objects:
        key = obj['Key']
        if product in key and key.endswith('grib.grb2'): #if key.endswith('.grb2'):
            local_file_path = os.path.join(download_dir, os.path.relpath(key, url_path))

            # Ensure the directory structure exists
            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

            # Download the file
            s3.download_file(bucket_name, key, local_file_path)
            num_files_downloaded += 1

            print(f"Downloaded: {key}")

In [5]:
def download_grb2_ncei(product, url_path, download_dir):

    # File counter
    num_files_downloaded = 0

    try:
        response = urllib.request.urlopen(url_path)
        html_content = response.read().decode('utf-8')
        soup = BeautifulSoup(html_content, 'html.parser')
        links = soup.find_all('a', href=lambda href: href and href.startswith(product) and href.endswith('grib.grb2'))
        
        for link in links:
            file_url = url_path + link['href']
            filename = link['href'].split('/')[-1]
            file_path = os.path.join(download_dir, filename)
            urllib.request.urlretrieve(file_url, file_path)
            print(f"Downloaded: {filename}")

            num_files_downloaded += 1

    except Exception as e:
        print(f"An error occurred: {e}")

Function to open a csv and if it does not already exist, create it and write the header to the top row.

In [6]:
def open_csv(filename, mask_variables):
    # Check if the file exists, and if not, write the header
    if not os.path.exists(filename):
        csvfile = open(filename, mode='a', newline='')
        writer = csv.writer(csvfile)
        writer.writerow(['cfs_run', 'forecast_year', 'forecast_month'] + mask_variables)
        print(f"{filename} created.")
    else:
        csvfile = open(filename, mode='a', newline='')
        writer = csv.writer(csvfile)
        print(f"{filename} opened.")
    return csvfile, writer

Function to grab a specific list of files based on the prefix or suffix of a file (ie. 'pgb', '.grb2')

In [7]:
def get_files(directory, affix, identifier):
    """
    Get a list of all GRIB2 files in the specified directory.

    Parameters:
    - directory (str): Path to the directory containing files.
    - affix (str): 'prefix' or 'suffix'
    - identifier (str):  (ie. 'pgb', 'flx', '.grb2', or '.nc')
    Returns:
    - List of file paths to the GRIB2 files.
    """
    files = []
    for file_name in os.listdir(directory):
        if affix == 'suffix': # ends with
            if file_name.endswith(identifier):
                file_path = os.path.join(directory, file_name)
                files.append(file_path)
        elif affix == 'prefix': # begins with
            if file_name.startswith(identifier):
                file_path = os.path.join(directory, file_name)
                files.append(file_path)
    sorted_files = sorted(files)
    return sorted_files

In [8]:
def create_directory(directory):
    """Create a directory if it doesn't already exist."""
    try:
        # Check if the directory already exists
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Directory '{directory}' created.")
        else:
            print(f"Directory '{directory}' already exists.")
    except Exception as e:
        print(f"An error occurred: {e}")

Function to delete the directory with CFS grb2 files because they are not needed after calculations are saved in the CSVs.

In [9]:
def delete_directory(directory_path):
    # Check if the directory exists
    if not os.path.isdir(directory_path):
        print(f"The directory {directory_path} does not exist.")
        return
    try:
        # Remove the entire directory tree
        shutil.rmtree(directory_path)
        print(f"Successfully deleted the directory and all its contents: {directory_path}")
    except Exception as e:
        print(f"Error deleting {directory_path}: {e}")

In [10]:
def check_url_exists(url):
    try:
        response = requests.head(url)
        # Check if the response is OK (status code 200)
        return response.status_code == 200
    except requests.RequestException:
        return False

Function to calculate the grid cell areas [m2] based on the mask file. This is needed to calculate total precipitation and evaporation because the units are [kg/m2].

In [11]:
def calculate_grid_cell_areas(lon, lat):
    # Calculate grid cell areas
    # Assuming lat and lon are 1D arrays
    # Convert latitude to radians

    R = 6371000.0  # Radius of Earth in meters
    lat_rad = np.radians(lat)

    # Calculate grid cell width in radians
    dlat = np.radians(lat[1] - lat[0])
    dlon = np.radians(lon[1] - lon[0])

    # Calculate area of each grid cell in square kilometers
    area = np.zeros((len(lat), len(lon)))
    for i in range(len(lat)):
        for j in range(len(lon)):
            area[i, j] = R**2 * dlat * dlon * np.cos(lat_rad[i])

    return area

Function to calculate evaporation based on the 2m air temperature and latent heat flux.

In [12]:
def calculate_evaporation(temperature_K, latent_heat):
    # ET = kg/(m^2*time^1) or 1 mm
    # LE = MJ/(M^2*time^1)
    # λ  = MJ/kg

    # Latent heat of vaporization varies slightly with temperature. Allen et al. (1998) provides an equation 
    # for calculating λ with air  temperature variation. Temperature in this case must be in degrees Celcius.

    # λ=2.501−(2.361×10−3)×Temp Celcius

    # so for our data with Temp in Kelvin...

    # λ=2.501−((2.361×10−3)×(Temp-273.15))

    # Our variable_lhf is in W/m^2 or J/(m^2*time^1). In order to convert to MJ we must multiply by 10^-6 or 
    # 0.000001. Now we have lamba and variable_lhf both in terms of MJ.

    # Equation below will provide an evaporation rate in kg/m2 per s. 

    lamda=(2.501-(0.002361*(temperature_K-273.15)))
    evaporation_rate=((latent_heat)*0.000001)/lamda

    return evaporation_rate # kg/m2 per s

Function to open each of the grib2 files and calculate the total precipitation, total evaporation, and average 2m air temperature over an entire basin, land, or lake for each of the Great Lakes. This uses the mask file to calculate each of these.

In [ ]:
def process_grib_files(download_dir, tmp_writer, evap_writer, pcp_writer, mask_lat, mask_lon, mask_ds, mask_variables, area):
    # Find all the .grb2 files in the directory
    file_list = get_files(download_dir, 'suffix', '.grb2')

    for grib2_file in file_list:

        filename = os.path.basename(grib2_file)
        parts = filename.split('.')
        cfs_run = parts[2]
        date_part = parts[3]  # Assuming parts[2] is in the format YYYYMM
        forecast_year = date_part[:4]
        forecast_month = date_part[4:6]

        # Check to see if the index file already exists and if it does, delete it
        index_file = grib2_file + '*.idx'
        if os.path.exists(index_file):
            print(f"Removing old index files.")
            os.remove(index_file)

        if filename.startswith('flxf'):

            ## 2 m Temperature ##
            # Create a temporary row to store data
            tmp_row = [cfs_run, forecast_year, forecast_month]

            # Open the flx file at the 2m level to pull the 2m air temperature
            flx_2mabove = cfgrib.open_dataset(grib2_file, engine='cfgrib', filter_by_keys={'typeOfLevel': 'heightAboveGround', 'level': 2}, decode_timedelta=False)
            #mean2t = flx_2mabove['mean2t']
            mean2t = flx_2mabove['avg_2t']

            # Cut the variable to the mask domain
            mean2t_cut = mean2t.sel(
                latitude=slice(mask_lat.max(), mask_lat.min()),
                longitude=slice(mask_lon.min(), mask_lon.max())
            )
            # Remap and upscale the variable to match the mask domain
            mean2t_remap = mean2t_cut.interp(latitude=mask_lat, longitude=mask_lon, method='linear')
            
            # Calculate mean2t for each of the mask variables (i.e., eri_lake, eri_basin, etc.)
            for mask_var in mask_variables:

                # Have to change the mask from fractional to all 1s and 0s
                mask_tmp = mask_ds.variables[mask_var][:]
                mask = np.ma.masked_where(np.isnan(mask_tmp), np.ones_like(mask_tmp))

                # Take the mean over the mask area
                tmp_avg = np.mean(mean2t_remap * mask)

                # Add data to the row
                tmp_row.append(tmp_avg.data)
            
            # Save the entire row to the CSV file
            tmp_writer.writerow(tmp_row) 

            ## Evaporation ##
            # Create a temporary row to store data
            evap_row = [cfs_run, forecast_year, forecast_month]

            # Open the flx file again but at the surface level to pull the latent heat flux
            flx_surface = cfgrib.open_dataset(grib2_file, engine='cfgrib', filter_by_keys={'typeOfLevel': 'surface'}, decode_timedelta=False)
            #mslhf = flx_surface['mslhf']
            mslhf = flx_surface['avg_slhtf']
            
            # Cut the variable to the mask domain
            mslhf_cut = mslhf.sel(
                latitude=slice(mask_lat.max(), mask_lat.min()),
                longitude=slice(mask_lon.min(), mask_lon.max())
            )
            # Remap and upscale the variable to match the mask domain
            mslhf_remap = mslhf_cut.interp(latitude=mask_lat, longitude=mask_lon, method='linear')

            # Calculate evaporation across the entire domain using air temp and latent heat flux
            evap = calculate_evaporation(mean2t_remap, mslhf_remap)
            
            year = int(forecast_year)
            month = int(forecast_month)

            _, num_days = calendar.monthrange(year, month)

            # Calculate evaporation for each of the mask variables (i.e., eri_lake, eri_land, etc.)
            for mask_var in mask_variables:
                
                mask = mask_ds.variables[mask_var][:]
                total_evap = (np.sum(evap * area * mask)) * num_days * 86400 # total in kg/sm2 * num of seconds in a month = kg/m2 or mm
                evap_mm = total_evap / np.sum(mask * area)

                # Add data to the row
                evap_row.append(evap_mm.data)

            # Save the entire row to the CSV file
            evap_writer.writerow(evap_row)

        ## Precipitation ##

        elif filename.startswith('pgbf'):
            # Create a temporary row to store data
            pcp_row = [cfs_run, forecast_year, forecast_month]

            # Open the pgb file at the surface level to pull the precipitation
            pgb_surface = cfgrib.open_dataset(grib2_file, engine='cfgrib', filter_by_keys={'typeOfLevel': 'surface'}, decode_timedelta=False)
  
            pcp = pgb_surface['tp']  # Total precipitation
            
            # Cut the variable to the mask domain
            pcp_cut = pcp.sel(
                latitude=slice(mask_lat.max(), mask_lat.min()),
                longitude=slice(mask_lon.min(), mask_lon.max())
            )
            # Remap and upscale the variable to match the mask domain
            pcp_remap = pcp_cut.interp(latitude=mask_lat, longitude=mask_lon, method='linear')

            year = int(forecast_year)
            month = int(forecast_month)

            _, num_days = calendar.monthrange(year, month)

            for mask_var in mask_variables:
                mask = mask_ds.variables[mask_var][:]
                
                total_pcp = (np.sum(pcp_remap * mask * area)) * 4 * num_days # pcp is for 6 hours - convert to monthly
                pcp_mm = total_pcp / np.sum(mask * area) # kg/m2 or mm

                # Add data to the row
                pcp_row.append(pcp_mm.data)

            # Save the entire row to the CSV file
            pcp_writer.writerow(pcp_row)

        # Delete index file to avoid issues in the future if grib files are reopened
        os.remove(index_file)
        print(f'Done with {filename}')

## Begin Script

Create the directory to download files to if it does not already exist

In [14]:
create_directory(download_dir)

Directory '/Users/fitzpatrick/Desktop/Data/CFS/' already exists.


Open the csv files or create new ones

In [15]:
tmp_csv, tmp_writer = open_csv(tmp_file, mask_variables)
evap_csv, evap_writer = open_csv(evap_file, mask_variables)
pcp_csv, pcp_writer = open_csv(pcp_file, mask_variables)

/Users/fitzpatrick/Desktop/Data/cnbs/CFS_TMP_forecasts_Avgs_K_9999.csv opened.
/Users/fitzpatrick/Desktop/Data/cnbs/CFS_EVAP_forecasts_Avgs_MM_9999.csv opened.
/Users/fitzpatrick/Desktop/Data/cnbs/CFS_PCP_forecasts_Avgs_MM_9999.csv opened.


This section allows the user to set the script to auto. When auto = yes, the script opens one of the CSVs (temperature), reads the last date that it recorded and automatically makes the start date the next day. It then will run through yesterday's date in order to be caught up. If auto = no, then the user can input a date range. This option is convienent for testing or for starting new CSVs.

In [18]:
if auto == 'yes':
    last_cfs = tmp_csv['cfs_run'].astype(str).iloc[-1][:8]
    start_date_i = datetime.strptime(last_cfs, '%Y%m%d') + timedelta(days=1)
    # Pull all the forecasts days up to yesterday (the most complete forecast)
    end_date_i = datetime.now() - timedelta(days=1)

else:
    start_date_i = datetime.strptime(start_date, "%Y-%m-%d") # User input from above
    end_date_i = datetime.strptime(end_date, "%Y-%m-%d") # User input from above

    # Check if start_date is equal to or after end_date
    if start_date_i == end_date_i:
        print("The csv files are up-to-date.")
        exit()  # Stop the script
    elif start_date_i > end_date_i:
        print("There is an error in the input dates. Please try again.")
    else:
        print(f"Starting from: {start_date_i.strftime('%Y-%m-%d')} and continuing through: {end_date_i.strftime('%Y-%m-%d')}")

# Create a date range
date_array = pd.date_range(start=start_date_i, end=end_date_i)

Starting from: 2024-08-13 and continuing through: 2025-02-01


Open the mask file. Pull the latitude and longitude to be used to cut the global variable down to just the Great Lakes domain and upscale. Also calculates area of each of the grid cells.

In [19]:
# Open the mask file and calculate the grid cell areas
mask_ds = nc.Dataset(mask_file)
mask_lat = mask_ds.variables['latitude'][:]
mask_lon = mask_ds.variables['longitude'][:]
area = calculate_grid_cell_areas(mask_lon, mask_lat)

The next cell can be skipped but is here to varify variable names.

In [20]:
import cfgrib

grib2_file = '/Users/fitzpatrick/Desktop/Data/CFS/20240817/CFS/flxf.01.2024081700.202408.avrg.grib.grb2'
flx_2mabove = cfgrib.open_dataset(grib2_file, engine='cfgrib', filter_by_keys={'typeOfLevel': 'surface'})

print(flx_2mabove)

Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240817/CFS/flxf.01.2024081700.202408.avrg.grib.grb2.5b7b6.idx' older than GRIB file


<xarray.Dataset> Size: 15MB
Dimensions:      (latitude: 190, longitude: 384)
Coordinates:
    time         datetime64[ns] 8B ...
    step         timedelta64[ns] 8B ...
    surface      float64 8B ...
  * latitude     (latitude) float64 2kB 89.28 88.34 87.4 ... -87.4 -88.34 -89.28
  * longitude    (longitude) float64 3kB 0.0 0.9375 1.875 ... 357.2 358.1 359.1
    valid_time   datetime64[ns] 8B ...
Data variables: (12/52)
    avg_utaua    (latitude, longitude) float32 292kB ...
    avg_vtaua    (latitude, longitude) float32 292kB ...
    avg_ishf     (latitude, longitude) float32 292kB ...
    avg_slhtf    (latitude, longitude) float32 292kB ...
    avg_t        (latitude, longitude) float32 292kB ...
    sdwe         (latitude, longitude) float32 292kB ...
    ...           ...
    snohf        (latitude, longitude) float32 292kB ...
    vbdsf        (latitude, longitude) float32 292kB ...
    vddsf        (latitude, longitude) float32 292kB ...
    nbdsf        (latitude, longitude) f

/opt/anaconda3/envs/cnbs/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:129: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


Begin loop to go through the user input dates. Loop creates a directory to download the CFS grib files, runs through the download_grb2_aws funtion to download and then run through the process_grib_files to do the calculations. It then saves the calculations to the CSV files, deletes the grib2 files and moves on to the next date.

In [21]:
for date in date_array:
    print(f"Beginning Files for {date}.")

    start_time = time.time()

    YYYY = date.strftime("%Y")
    YYYYMM = date.strftime("%Y%m")
    YYYYMMDD = date.strftime("%Y%m%d")

    date = date.strftime('%Y%m%d')
    download_path = f'{download_dir}{date}/CFS/'

    if not os.path.exists(download_path):
        os.makedirs(download_path)

    # Download the grib2 files using AWS or NCEI
    if download_cfs == 'yes':
        for utc_time in utc:
            for product in products:
                if source == 'aws':
                    url_path = f'cfs.{date}/{utc_time}/monthly_grib_01/'
                    download_grb2_aws(product, bucket_name, url_path, download_path)
                elif source == 'ncei':
                    base_url = 'https://www.ncei.noaa.gov/data/climate-forecast-system/access/operational-9-month-forecast/monthly-means/'
                    url_path = f'{base_url}/{YYYY}/{YYYYMM}/{YYYYMMDD}/{YYYYMMDD}{utc_time}/'
                    if not url_path or not check_url_exists(url_path):
                        print(f"No files available for {date}.")
                    else:
                        download_grb2_ncei(product, url_path, download_path)
                else:
                    print('Input source does not exist. Source must be aws or ncei.')
    
    if process_cfs == 'yes':
        print('process = yes')
        process_grib_files(download_path, tmp_writer, evap_writer, pcp_writer, mask_lat, mask_lon, mask_ds, mask_variables, area)

        if delete_files == 'yes':
            delete_directory(download_path)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    
    print(f'Done with {date}.')

Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240813/CFS/flxf.01.2024081300.202408.avrg.grib.grb2.da267.idx' older than GRIB file


Beginning Files for 2024-08-13 00:00:00.
process = yes


Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240813/CFS/flxf.01.2024081300.202408.avrg.grib.grb2.5b7b6.idx' older than GRIB file
Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240813/CFS/flxf.01.2024081300.202409.avrg.grib.grb2.da267.idx' older than GRIB file
Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240813/CFS/flxf.01.2024081300.202409.avrg.grib.grb2.5b7b6.idx' older than GRIB file


Done with flxf.01.2024081300.202408.avrg.grib.grb2


Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240813/CFS/flxf.01.2024081300.202410.avrg.grib.grb2.da267.idx' older than GRIB file
Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240813/CFS/flxf.01.2024081300.202410.avrg.grib.grb2.5b7b6.idx' older than GRIB file


...

Done with pgbf.01.2024081900.202505.avrg.grib.grb2


Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240819/CFS/pgbf.01.2024081906.202409.avrg.grib.grb2.5b7b6.idx' older than GRIB file


Done with pgbf.01.2024081906.202408.avrg.grib.grb2


Ignoring index file '/Users/fitzpatrick/Desktop/Data/CFS/20240819/CFS/pgbf.01.2024081906.202410.avrg.grib.grb2.5b7b6.idx' older than GRIB file


Done with pgbf.01.2024081906.202409.avrg.grib.grb2
Done with pgbf.01.2024081906.202410.avrg.grib.grb2
Done with pgbf.01.2024081906.202411.avrg.grib.grb2
Done with pgbf.01.2024081906.202412.avrg.grib.grb2
Done with pgbf.01.2024081906.202501.avrg.grib.grb2
Done with pgbf.01.2024081906.202502.avrg.grib.grb2
Done with pgbf.01.2024081906.202503.avrg.grib.grb2
Done with pgbf.01.2024081906.202504.avrg.grib.grb2
Done with pgbf.01.2024081906.202505.avrg.grib.grb2
Done with pgbf.01.2024081912.202408.avrg.grib.grb2
Done with pgbf.01.2024081912.202409.avrg.grib.grb2
Done with pgbf.01.2024081912.202410.avrg.grib.grb2
Done with pgbf.01.2024081912.202411.avrg.grib.grb2
Done with pgbf.01.2024081912.202412.avrg.grib.grb2
Done with pgbf.01.2024081912.202501.avrg.grib.grb2
Done with pgbf.01.2024081912.202502.avrg.grib.grb2
Done with pgbf.01.2024081912.202503.avrg.grib.grb2
Done with pgbf.01.2024081912.202504.avrg.grib.grb2
Done with pgbf.01.2024081912.202505.avrg.grib.grb2
Done with pgbf.01.2024081918.20

Close any open files before finishing script.

In [14]:
mask_ds.close()